In [1]:
#### run imports, configure pax and hax
%run "Initialize.ipynb"
from pax import units, configuration, datastructure
pax_config = configuration.load_configuration('XENON1T')
n_channels = pax_config['DEFAULT']['n_channels']
pmts = pax_config['DEFAULT']['pmts']
tpc_height = pax_config['DEFAULT']['tpc_length']
tpc_radius = pax_config['DEFAULT']['tpc_radius']
gains = pax_config['DEFAULT']['gains']
# get channel number of busy on
busy_on_ch = pax_config['DEFAULT']['channels_in_detector']['busy_on'][0]

import hax
from hax import cuts

hax.__version__
hax.init(experiment='XENON1T',
         raw_data_access_mode='local',
         raw_data_local_path='/project/lgrandi/xenon1t/raw',
        main_data_paths=['/project/lgrandi/xenon1t/processed/pax_v6.6.5' , 
                         '/project2/lgrandi/xenon1t/processed/pax_v6.6.5'], # add here correct path for AmBe
        #minitree_paths = ['/project*/lgrandi/xenon1t/minitrees', 
         #                 '/project*/lgrandi/xenon1t/minitrees/latest',
          #               '.', '/project*/lgrandi/xenon1t/minitrees/pax_v6.6.5'],
        #minitree_paths = ['/home/fieguth/DEC','*'],
         pax_version_policy= '6.6.5'  )
        #minitree_paths = ['/project*/lgrandi/xenon1t/minitrees/latest'],
         #pax_version_policy= '6.6.5'  )



datasets = hax.runs.datasets # this variable holds all dataset info

In [6]:
## Select datasets
#dsets = datasets[(datasets.source__type == 'none')| (datasets.source__type == 'None')]
dsets = hax.runs.tags_selection(include=['sciencerun1'], exclude=['bad', 'messy', '*trip', '*quake','test'])
dsets = dsets[(dsets.source__type == 'none')| (dsets.source__type == 'None')] # 
dsets = dsets[(dsets.location != '')]
#dsets['run_time_s'] = pd.to_timedelta(pd.to_datetime(dsets.end) - pd.to_datetime(dsets.start)).dt.seconds
#dsets = dsets[dsets.run_time_s > 3600] # select only runs of an hour or more, CHECK IF NECESSARY FOR AMBE
# some of the datasets had anomalously high background rates. I didn't use these datasets for the analysis.
# dsets['background_rate'] = dsets.trigger__events_built/dsets.run_time_s # background rate (avg. events/s)

# select data range
dsets = dsets[(dsets.start > pd.to_datetime('06/06/2017')) & (dsets.end < pd.to_datetime('06/13/2017'))]
#dsets = dsets[(dsets.start > pd.to_datetime('1/10/2017')) & (dsets.end < pd.to_datetime('1/18/2017'))]
dsets['start_date'] = dsets.start.dt.date

run_names = dsets.name

# get rid of problematic runs
bad_runs = ['161204_1517','170111_031','170111_0314'] # n'o minitree'

for run in bad_runs:
    run_names = run_names[run_names != run]
    dsets  = dsets[dsets.name != run]

# In order to create the batch jobs we don't need the run number (=index) instead of the time and date
run_numbers = dsets.index
print(len(run_numbers.values))

49


In [7]:
# The link to the original batch queue examples can be found here: https://github.com/XENON1T/XeAnalysisScripts/tree/master/examples/BatchMinitrees_Ted

# 2016-09, sanderb@nikhef.nl: from Chris with Sander's personalization

# log files will be created in the folder where you run this script
# For every job you will get an email, make sure this won't be blocked as it will look like spam if you get 500+ emails
# You need write permission for making the processing_dir folder

x = """#!/bin/bash
#SBATCH --job-name={run}
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=2000
#SBATCH --mail-type=ALL
#SBATCH --output=minitree_%J.log
#SBATCH --error=minitree_%J.log
#SBATCH --account=pi-lgrandi
#SBATCH --qos=xenon1t
#SBATCH --partition=xenon1t
#SBATCH --mail-user=<c.wittweg@wwu.de>
export PATH=/project/lgrandi/anaconda3/bin:$PATH
export PROCESSING_DIR=/home/wittweg/minitrees/minitree_{run}
        
mkdir -p ${{PROCESSING_DIR}}
cd ${{PROCESSING_DIR}}
rm -f pax_event_class*
source activate pax_head

echo python ..treebuilder.py {run}
python ../treebuilder.py {run}

mv *root /home/wittweg/minitrees/
"""

# Use submit procedure from CAX
from cax.qsub import submit_job

#Define which runs we want to process (max per submit is 500 jobs!)

#runs=[10181,10182]

# Make sure you do not submit too many jobs
if len(run_numbers.values) < 450:
    runs = run_numbers.values
    # For every run, make and submit the script
    for run in runs:
        y = x.format(run=run)    
        submit_job(y)
else:
    print('Too many files. Try again')

# Check your jobs with: 'qstat -u <username>'
# Check number of submitted jobs with 'qstat -u <username> | wc -l' (is off by +2 btw)

INFO:root:b'Submitted batch job 29617861\n'
INFO:root:b'Submitted batch job 29617862\n'
INFO:root:b'Submitted batch job 29617864\n'
INFO:root:b'Submitted batch job 29617866\n'
INFO:root:b'Submitted batch job 29617869\n'
INFO:root:b'Submitted batch job 29617870\n'
INFO:root:b'Submitted batch job 29617871\n'
INFO:root:b'Submitted batch job 29617872\n'
INFO:root:b'Submitted batch job 29617873\n'
INFO:root:b'Submitted batch job 29617874\n'
INFO:root:b'Submitted batch job 29617875\n'
INFO:root:b'Submitted batch job 29617878\n'
INFO:root:b'Submitted batch job 29617879\n'
INFO:root:b'Submitted batch job 29617880\n'
INFO:root:b'Submitted batch job 29617882\n'
INFO:root:b'Submitted batch job 29617883\n'
INFO:root:b'Submitted batch job 29617884\n'
INFO:root:b'Submitted batch job 29617885\n'
INFO:root:b'Submitted batch job 29617888\n'
INFO:root:b'Submitted batch job 29617889\n'
INFO:root:b'Submitted batch job 29617890\n'
INFO:root:b'Submitted batch job 29617891\n'
INFO:root:b'Submitted batch job 